# POS Tagging

### Premissa

Consideramos `um dataset com tags` e aplicamos `aprendizagem supervisionada`

### Fato

Essa tarefa envolve o aprendizado de `múltiplas classes`

In [25]:
# import nltk os corpus que vamos utilizar
import nltk
nltk.download('brown')
nltk.download('treebank')

# import das bibliotecas importantes
from nltk import word_tokenize
from nltk.corpus import brown as cb
from nltk.corpus import treebank as tb
import pprint
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package brown to /home/geanderson/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /home/geanderson/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


#### Brown Corpus

In [26]:
# vamos usar o brown corpus
raw_text = nltk.Text(cb.words('ca01'))
print (raw_text)

<Text: The Fulton County Grand Jury said Friday an...>


In [27]:
print (cb.words()[0:20])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that']


In [28]:
print (cb.tagged_words()[0:10])

[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN')]


In [29]:
tagged_sentences_brown_corpus = nltk.corpus.brown.tagged_sents()
pprint.pprint(tagged_sentences_brown_corpus[0])

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]


#### Penn-Treebank Corpus

In [30]:
raw_text = nltk.Text(tb.words()[0:10])
print (raw_text)

<Text: Pierre Vinken , 61 years old , will...>


In [31]:
print (tb.words()[0:10])

['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the']


In [32]:
tagged_sentences_treebank_corpus = nltk.corpus.treebank.tagged_sents()
pprint.pprint (tagged_sentences_treebank_corpus[0])

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]


In [33]:
print ("Sentenças com Tags: ", len(tagged_sentences_treebank_corpus))

Sentenças com Tags:  3914


In [34]:
print ("Palavras com Tags:", len(nltk.corpus.treebank.tagged_words()))

Palavras com Tags: 100676


#### Gerando as features

In [35]:
def features(sentence, index):
    return {
    'word': sentence[index],
    'is_first': index == 0,
    'is_last': index == len(sentence) - 1,
    'is_capitalized': sentence[index][0].upper() == sentence[index][0],
    'is_all_caps': sentence[index].upper() == sentence[index],
    'is_all_lower': sentence[index].lower() == sentence[index],
    'prefix-1': sentence[index][0],
    'prefix-2': sentence[index][:2],
    'prefix-3': sentence[index][:3],
    'suffix-1': sentence[index][-1],
    'suffix-2': sentence[index][-2:],
    'suffix-3': sentence[index][-3:],
    'prev_word': '' if index == 0 else sentence[index - 1],
    'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    'has_hyphen': '-' in sentence[index],
    'is_numeric': sentence[index].isdigit(),
    'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [36]:
pprint.pprint(features(['This', 'is', 'a', 'sentence'], 0))

{'capitals_inside': False,
 'has_hyphen': False,
 'is_all_caps': False,
 'is_all_lower': False,
 'is_capitalized': True,
 'is_first': True,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'is',
 'prefix-1': 'T',
 'prefix-2': 'Th',
 'prefix-3': 'Thi',
 'prev_word': '',
 'suffix-1': 's',
 'suffix-2': 'is',
 'suffix-3': 'his',
 'word': 'This'}


#### Transformando o dataset

Extraindo palavras de frases marcadas usando a função `untag`


In [37]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

#### Transforformar o dataset em X, Y pares onde X = features e Y = POS labels

In [38]:
def transform_to_dataset(tagged_sentences):
    X, y = [], []
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
    return X, y

#### Construir o training e testing dataset   

In [39]:
cutoff = int(.80 * len(tagged_sentences_treebank_corpus))
training_sentences = tagged_sentences_treebank_corpus[:cutoff]
test_sentences = tagged_sentences_treebank_corpus[cutoff:]

In [40]:
print (len(training_sentences))
print (len(test_sentences))

3131
783


In [41]:
X, y = transform_to_dataset(training_sentences)

In [42]:
print(len(X)) 
print(len(y))

80637
80637


####  Treinando o Modelo de Árvore de Decisão

In [43]:
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])

In [44]:
# Use apenas as primeiras 10 mil amostras se estiver executando várias vezes
clf.fit(X[:20000],y[:20000]) 

Pipeline(memory=None,
     steps=[('vectorizer', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)), ('classifier', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

#### Mensurando a Acurácia

In [45]:
X_test, y_test = transform_to_dataset(test_sentences)

In [46]:
print ("Accuracy:{:.3%}".format(clf.score(X_test, y_test)))

Accuracy:91.606%


### Gerar POS tags para sentenças

In [47]:
def pos_tag(sentence):
    tagged_sentence = []
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    return zip(sentence, tags)

In [48]:
POS_list = list(pos_tag(word_tokenize('This is my friend, John.')))
for t in POS_list:
    print(u"{:<16}{:>2}".format(str(t[0]),str(t[1])))   

This            DT
is              VBZ
my              PRP$
friend          NN
,                ,
John            NNP
.                .


In [49]:
POS_list = list(pos_tag(word_tokenize("We will meet at eight o'clock on Thursday morning.")))
for t in POS_list:
    print(u"{:<16}{:>2}".format(str(t[0]),str(t[1])))   

We              IN
will            MD
meet            VB
at              IN
eight           NN
o'clock         NN
on              IN
Thursday        NNP
morning         NN
.                .


In [50]:
POS_list = list(pos_tag(word_tokenize('Alexander, the great...!')))
for t in POS_list:
    print(u"{:<16}{:>2}".format(str(t[0]),str(t[1])))   

Alexander       JJR
,                ,
the             DT
great           NN
...              :
!               CD


In [51]:
POS_list = list(pos_tag(word_tokenize('Alexander the Great, was a king of the ancient Greek kingdom of Macedon.')))
for t in POS_list:
    print(u"{:<16}{:>2}".format(str(t[0]),str(t[1])))   

Alexander       IN
the             DT
Great           NNP
,                ,
was             VBD
a               DT
king            NN
of              IN
the             DT
ancient         DT
Greek           NNP
kingdom         NN
of              IN
Macedon         NNP
.                .


#### Fim!